# TOPSIS法
TOPSIS法,又称优劣解距离法
评价类类型的一种模型算法，可结合层次分析法添加权重计算
## 层次分析法的局限性
1. 决策层不能太多，太多则n会很大，判断矩阵和一致性矩阵的差距就会很大
2. 决策层指标通常需要自己确定，当给定指标时，需要利用给定数据使评价更加准确
## TOPSIS法步骤
1. 数据处理
2. 原始矩阵正向化
3. 正向化矩阵标准化
4. 计算得分并归一化

## 数据处理

In [3]:
#导入所需库
import numpy as np #矩阵计算
import pandas as pd #数据处理

In [12]:
#读取excel文件
data = pd.read_excel('20条河流的水质情况数据.xlsx', sheet_name='Sheet1') #读取数据
print(data) #打印原始数据,为dataframe格式

   河流  含氧量（ppm)   PH值  细菌总数(个/mL)  植物性营养物量（ppm)
0   A      4.69  6.59          51         11.94
1   B      2.03  7.86          19          6.46
2   C      9.11  6.31          46          8.91
3   D      8.61  7.05          46         26.43
4   E      7.13  6.50          50         23.57
5   F      2.39  6.77          38         24.62
6   G      7.69  6.79          38          6.01
7   H      9.30  6.81          27         31.57
8   I      5.45  7.62           5         18.46
9   J      6.19  7.27          17          7.51
10  K      7.93  7.53           9          6.52
11  L      4.40  7.28          17         25.30
12  M      7.46  8.24          23         14.42
13  N      2.01  5.55          47         26.31
14  O      2.04  6.40          23         17.91
15  P      7.73  6.14          52         15.72
16  Q      6.35  7.58          25         29.46
17  R      8.29  8.41          39         12.02
18  S      3.54  7.27          54          3.16
19  T      7.44  6.26           8       

## 第一步 原始矩阵正向化(三个函数)

In [13]:
# 极小型转为极大型指标
def dataDirection_1(datas):
    numpy_array = np.array(datas)  # 将数据转换为numpy数组
    max_value = numpy_array.max()  # 计算最大值
    def normalization(data):
        return max_value-data  # 使用max - x 来转换极小型指标为极大型指标 

    return [normalization(data) for data in datas]#使用列表推导式来处理数据


# 中间型指标转为极大型指标
def dataDirection_2(datas, best):
    numpy_array  = np.array(datas)  # 将数据转换为numpy数组
    numpy_array = numpy_array - best  # 减去最佳值
    numpy_array = np.abs(numpy_array)  # 取绝对值
    M = numpy_array.max()  # 计算最大值
    # 防止除以零的情况
    if M == 0:
        return [1] * len(datas)  # 如果最大值为 0，返回所有值为 1（表示所有值相等，最大值）
    
    def normalization(data):
        result = 1 - (abs(data - best) / M)  # 计算极大型指标
        return result if result >= 0 else 0
    # 处理数据，避免负值

    return [normalization(data) for data in datas]


# 区间型指标转为极大型指标
def dataDirection_3(datas,x_min,x_max):
    numpy_array = np.array(datas)  # 将数据转换为numpy数组
    M = max(x_min-numpy_array.max(),numpy_array.max()-x_max) #计算M(详见笔记)
    def normalization(data):
        if data < x_min:
            return 1 - (x_min - data) / M
        elif data > x_max:
            return 1 - (data - x_max) / M
        else:
            return 1

    return [normalization(data) for data in datas]

In [28]:
#例题数据处理
#对于本例题，PH值 细菌总数(个/mL) 植物性营养物量（ppm)
#三列需要进行正向化处理，分别为中间型(best = 7)，极小型和区间型指标(10-20)

#中间型指标(PH值)转为极大型指标
ph_list = dataDirection_2(data.loc[:,"PH值"], 7)
ph_array = np.array(ph_list)
ph_4f = np.round(ph_array, 6)
#print("PH值的极大型指标为：",hanyang_4f)

#极小型型指标(细菌总数)转为极大型指标
xijun_list = dataDirection_1(data.loc[:,"细菌总数(个/mL)"])
xijun_array = np.array(xijun_list)
xijun_4f = np.round(xijun_array, 6)
#print("细菌总数(个/mL)的极大型指标为：",xijun_4f)

#区间型指标(植物性营养物量)转为极大型指标
yingyangwu_list = dataDirection_3(data.loc[:,"植物性营养物量（ppm)"], 10, 20)
yingyangwu_array = np.array(yingyangwu_list)
yingyangwu_4f = np.round(yingyangwu_array, 6)
#print("植物性营养物量（ppm)的极大型指标为：",yingyangwu_4f)

# 使用 column_stack 拼接为二维矩阵
hanyang_array = np.array(data.loc[:,"含氧量（ppm)"])  # 将数据转换为numpy数组
matrix = np.column_stack((hanyang_array,ph_4f, xijun_4f, yingyangwu_4f))
print(matrix)


[[4.69000e+00 7.17241e-01 3.00000e+00 1.00000e+00]
 [2.03000e+00 4.06897e-01 3.50000e+01 6.94036e-01]
 [9.11000e+00 5.24138e-01 8.00000e+00 9.05791e-01]
 [8.61000e+00 9.65517e-01 8.00000e+00 4.44252e-01]
 [7.13000e+00 6.55172e-01 4.00000e+00 6.91443e-01]
 [2.39000e+00 8.41379e-01 1.60000e+01 6.00691e-01]
 [7.69000e+00 8.55172e-01 1.60000e+01 6.55143e-01]
 [9.30000e+00 8.68966e-01 2.70000e+01 0.00000e+00]
 [5.45000e+00 5.72414e-01 4.90000e+01 1.00000e+00]
 [6.19000e+00 8.13793e-01 3.70000e+01 7.84788e-01]
 [7.93000e+00 6.34483e-01 4.50000e+01 6.99222e-01]
 [4.40000e+00 8.06897e-01 3.70000e+01 5.41919e-01]
 [7.46000e+00 1.44828e-01 3.10000e+01 1.00000e+00]
 [2.01000e+00 0.00000e+00 7.00000e+00 4.54624e-01]
 [2.04000e+00 5.86207e-01 3.10000e+01 1.00000e+00]
 [7.73000e+00 4.06897e-01 2.00000e+00 1.00000e+00]
 [6.35000e+00 6.00000e-01 2.90000e+01 1.82368e-01]
 [8.29000e+00 2.75860e-02 1.50000e+01 1.00000e+00]
 [3.54000e+00 8.13793e-01 0.00000e+00 4.08816e-01]
 [7.44000e+00 4.89655e-01 4.600

## 第二步 正向化矩阵标准化

In [ ]:
# 正向矩阵标准化函数
def standardize_matrix(X):
    squared = np.square(X)  # 平方
    column_sums = np.sum(squared, axis=0) # 按列求和
    sqrt_sums = np.sqrt(column_sums)  # 开平方
    # 防止除以零的情况
    sqrt_sums[sqrt_sums == 0] = 1  # 如果某列的平方和为 0，则将其设置为 1（避免除零）
    normalized = X / sqrt_sums  # 除以平方和的平方根
    return normalized  # 返回标准化后的矩阵

# 标准化矩阵
standardized_matrix = standardize_matrix(matrix)  # 调用标准化函数
print("标准化后的矩阵：",standardized_matrix)  # 打印标准化后的矩阵

标准化后的矩阵： [[0.16218592 0.24825515 0.02454403 0.30645758]
 [0.07019987 0.14083729 0.28634707 0.2126926 ]
 [0.3150349  0.18141734 0.06545076 0.27758652]
 [0.29774429 0.33418972 0.06545076 0.13614439]
 [0.24656409 0.22677151 0.03272538 0.21189795]
 [0.08264911 0.29122243 0.13090152 0.18408631]
 [0.26592957 0.29599654 0.13090152 0.20077354]
 [0.32160534 0.30077099 0.22089631 0.        ]
 [0.18846764 0.19812688 0.4008859  0.30645758]
 [0.21405774 0.28167422 0.30270976 0.24050423]
 [0.27422907 0.21961052 0.36816052 0.21428188]
 [0.15215736 0.27928735 0.30270976 0.16607519]
 [0.25797589 0.05012861 0.25362169 0.30645758]
 [0.06950825 0.         0.05726941 0.13932297]
 [0.07054569 0.20290099 0.25362169 0.30645758]
 [0.26731282 0.14083729 0.01636269 0.30645758]
 [0.21959074 0.20767509 0.237259   0.05588806]
 [0.2866783  0.00954821 0.12272017 0.30645758]
 [0.12241751 0.28167422 0.         0.12528476]
 [0.25728427 0.16948191 0.37634187 0.0836997 ]]


## 第三步 计算得分并归一化

In [32]:
#计算得分并归一化(暂时不考虑权重)
def calculate_scores(Z):
    # 计算每一行的平方和
    Z_plus = np.max(Z, axis=0)#  # 计算每一列的最大值(正理想解)
    Z_minus = np.min(Z, axis=0)  # 计算每一列的最小值(负理想解)

    # 3. 计算每个方案到正/负理想解的欧式距离
    D_plus = np.sqrt(np.sum((Z - Z_plus) ** 2, axis=1))
    D_minus = np.sqrt(np.sum((Z - Z_minus) ** 2, axis=1))
    # 4. 计算相对贴近度 Si
    S = D_minus / (D_plus + D_minus)
    S = S/ np.sum(S)  # 归一化处理
    return S  # 返回每个方案归一化猴儿的得分

# 计算得分
scores = calculate_scores(standardized_matrix)  # 调用计算得分函数
print("得分：", scores)  # 打印正理想解

得分： [0.04505806 0.04779878 0.0484974  0.0488202  0.04311309 0.04483782
 0.05394726 0.05099837 0.06807384 0.0683933  0.07016174 0.05909345
 0.05265186 0.01919585 0.05330661 0.04335863 0.04657436 0.04384359
 0.03580976 0.05646602]
